In [5]:
import pandas as pd
import numpy as np

path = ("ECAI")
folder = lambda db, eff, we : f"/db{db}_effrate{eff}_we{we}_ECAI"

In [6]:
data = pd.read_csv(path + folder(0, 0.2, 0)*2 + "/t2s.csv").to_numpy()
survival = data.copy()
survival[data > 0] = 1
survival[np.isnan(data)] = 0

print(np.nanmedian(survival, axis=0))
print(np.nanpercentile(survival, 75, axis=0)- np.nanpercentile(survival, 25, axis=0))

[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]


In [7]:
dbs = [0, 1, 10, 100]
effs = [0, 0.2, 0.6, 1]

survivor_database =  {eff: {db : {we : pd.DataFrame() for we in [0, 10]} for db in dbs} for eff in effs}
t2s_database = {eff: {db : {we : pd.DataFrame() for we in [0, 10]} for db in dbs} for eff in effs}

for db in dbs:
    for eff in effs:
        for we in [0, 10]:
            try:
                data = pd.read_csv(path + folder(db, eff, we)*2 + "/t2s.csv").to_numpy()
            except FileNotFoundError:
                continue
            survival = data.copy()
            survival[data > 0] = 1
            survival[np.isnan(data)] = 0
            survivor_database[eff][db][we] = pd.DataFrame(survival, columns=[f"Survival_Ag{i}" for i in range(1, 6)])
            t2s_database[eff][db][we] = pd.DataFrame(data, columns=[f"T2S_Ag{i}" for i in range(1, 6)])


In [8]:
t2s_database[0.2][1][10].describe()

,T2S_Ag1,T2S_Ag2,T2S_Ag3,T2S_Ag4,T2S_Ag5
count,4999.000000,4620.000000,1757.000000,4555.000000,3627.000000
mean,167.996399,373.728788,440.107001,367.517892,409.352633
std,47.118494,60.624478,45.601946,63.640765,54.406576
min,66.000000,195.000000,256.000000,184.000000,222.000000
25%,135.000000,329.000000,412.000000,321.000000,370.000000
50%,160.000000,373.000000,451.000000,365.000000,413.000000
75%,193.000000,418.000000,475.000000,415.000000,453.000000
max,420.000000,500.000000,500.000000,500.000000,500.000000


In [50]:
row = 0.6
grouped_values = np.zeros((4,2))
grouped_values_std = np.zeros((4,2))
grouped_values_median = np.zeros((4,2))
grouped_values_iqr = np.zeros((4,2))
for k, db in enumerate(dbs):
    value = t2s_database[row][db][10].to_numpy()
    grouped_values[k, 0] = np.nanmean(value[:,:int(row*5)])
    grouped_values_std[k, 0] = np.nanstd(value[:,:int(row*5)])
    grouped_values_median[k, 0] = np.nanmedian(value[:,:int(row*5)])
    grouped_values_iqr[k, 0] = np.nanpercentile(value[:,:int(row*5)], 75) - np.nanpercentile(value[:,:int(row*5)], 25)
    grouped_values[k, 1] = np.nanmean(value[:,int(row*5)+1:])
    grouped_values_std[k, 1] = np.nanstd(value[:,int(row*5)+1:])
    grouped_values_median[k, 1] = np.nanmedian(value[:,int(row*5)+1:])
    grouped_values_iqr[k, 1] = np.nanpercentile(value[:,int(row*5)+1:], 75) - np.nanpercentile(value[:,int(row*5)+1:], 25)
    
print(grouped_values_iqr)

[[119.  28.]
 [ 72.  46.]
 [ 77.  45.]
 [ 68.  43.]]


In [10]:
grouped_values_median

array([[149., 448.],
       [160., 401.],
       [147., 383.],
       [151., 393.]])

In [11]:
row = 0.2
for k, db in enumerate(dbs):
    survivor_np = survivor_database[row][db][10].to_numpy()
    print(f"DB: {db}", survivor_np.mean(axis=0).round(2)*100)
print("--------------")
for k, db in enumerate(dbs):
    survivor_np = survivor_database[row][db][10].to_numpy()
    print(f"DB: {db}", survivor_np.std(axis=0).round(2)*100)
print("--------------")  
for k, db in enumerate(dbs):
    survivor_np = survivor_database[row][db][10].to_numpy()
    print(f"DB: {db}",np.median(survivor_np, axis=0)*100)

DB: 0 [87.  0.  5.  8.  1.]
DB: 1 [100.  92.  35.  91.  73.]
DB: 10 [100.  95.  95.  95.  94.]
DB: 100 [97. 85. 72. 84. 49.]
--------------
DB: 0 [34.  6. 22. 27.  7.]
DB: 1 [ 1. 26. 48. 28. 45.]
DB: 10 [ 0. 22. 22. 23. 24.]
DB: 100 [18. 36. 45. 37. 50.]
--------------
DB: 0 [100.   0.   0.   0.   0.]
DB: 1 [100. 100.   0. 100. 100.]
DB: 10 [100. 100. 100. 100. 100.]
DB: 100 [100. 100. 100. 100.   0.]


In [36]:
# Print avg + std of grouped survivor
for row in [0.2, 0.6]:
    g = []
    for db in [0, 1, 10]:
        data = survivor_database[row][db][10].to_numpy()
        grouped_survivors = [data[:, :int(row*5)].flatten(), data[:, int(row*5)+1:].flatten()]
        g.append(grouped_survivors)
    
    print(" & ".join([f"{g[i][1].mean().round(2)*100}\% \\pm {g[i][1].std().round(2)*100}\% "for i in range(len([0, 1, 10]))]), "\\\\")


4.0\% \pm 20.0\%  & 66.0\% \pm 47.0\%  & 94.0\% \pm 23.0\%  \\
0.0\% \pm 3.0\%  & 100.0\% \pm 2.0\%  & 100.0\% \pm 1.0\%  \\


In [38]:
# Print avg + std of grouped survivor
for row in [0.2]:
    agents = []
    for db in [0, 1, 10]:
        data = survivor_database[row][db][10].to_numpy()
        print(f"DB = {db} &"," & ".join([f"{data.mean(axis=0)[ag].round(2)*100}\% \\pm {data.std(axis=0)[ag].round(2)*100}\% "for ag in range(5)]), "\\\\")

DB = 0 & 87.0\% \pm 34.0\%  & 0.0\% \pm 6.0\%  & 5.0\% \pm 22.0\%  & 8.0\% \pm 27.0\%  & 1.0\% \pm 7.000000000000001\%  \\
DB = 1 & 100.0\% \pm 1.0\%  & 92.0\% \pm 26.0\%  & 35.0\% \pm 48.0\%  & 91.0\% \pm 28.000000000000004\%  & 73.0\% \pm 45.0\%  \\
DB = 10 & 100.0\% \pm 0.0\%  & 95.0\% \pm 22.0\%  & 95.0\% \pm 22.0\%  & 95.0\% \pm 23.0\%  & 94.0\% \pm 24.0\%  \\
